In [122]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
import json
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup


In [143]:
# 연재 중 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

weekday = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

for day in weekday:
    week_data_url = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/' + day + '?timeStamp='
    soup = readHtml(week_data_url)
    str_soup = str(soup)
    dict_soup = json.loads(str_soup)
    
    for toon in dict_soup['data']:
        toon_id = 'daum_' + str(toon['id'])
        toon_url = 'http://webtoon.daum.net/data/pc/webtoon/view/' + toon['nickname'] + '?timeStamp'
        toon_name = toon['title']  
        toon_thumb_url = toon['pcThumbnailImage']['url']
        toon_site = 'daum'
        toon_desc = toon['introduction'].replace("  ", " ")
        
        # 완결 여부
        if toon['finishYn'] == 'Y':
            is_end = 'O'
        else:
            is_end = 'X'
        
        # 작가 이름
        wrt_name_list = list()
        for names in toon['cartoon']['artists']:
            wrt_name_list.append(names['name'])
        wrt_name_list = list(set(wrt_name_list))
        wrt_name = '/'.join(wrt_name_list)
        
        # 요일 정보
        for days in toon['webtoonWeeks']:
            cursor1.execute(weekday_sql, (toon_id, days['weekDay']))
        
        # 장르 정보
        for genres in toon['cartoon']['genres']:
            genre_name = genres['name'].replace('순정', '로맨스').replace('코믹', '개그').replace('공포', '스릴러')
            cursor2.execute(genre_sql, (toon_id, genre_name))
        
        # toon_info
        cursor3.execute(sel_sql)
        result = cursor3.fetchall()
        if (toon_id not in result):
            cursor4.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url))
        
        db.commit()
    
db.close()










C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1716-스릴러' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1705-스릴러' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-mon' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-sat' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-개그' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-액션' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\curso

In [124]:
### 회차정보 크롤링 ###
#소연아 왜 안했어

#def episode_crawl(toon_data_url):
sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "daum"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                        values (%s, %s, %s, %s, %s)'

toon_data_url = 'http://webtoon.daum.net/data/pc/webtoon/view/subwayline1?timeStamp'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
    
soup = readHtml(toon_data_url)
str_soup = str(soup)
dict_soup = json.loads(str_soup)
episode = dict_soup['data']['webtoon']['webtoonEpisodes']
print(episode)
    

[{'id': 22444, 'episode': 2, 'title': '1화', 'shortTitle': '1화', 'thumbnailImage': {'id': 22444, 'url': 'http://i1.cartoon.daumcdn.net/svc/image/U03/cartoon/52364C60013C2F0001', 'name': None, 'size': None, 'width': 0, 'height': 0, 'mediaType': None, 'encryptKey': None, 'serviceStatus': 'service'}, 'episodeImage': None, 'encryptUseYn': 'Y', 'serviceStatus': 'service', 'articleId': 22444, 'commentUseYn': 'Y', 'dateCreated': '20190123000000', 'webtoon': None, 'serviceType': 'free', 'multiType': None, 'multiBgm': None, 'multiBackgroundImage': None, 'price': 0, 'padtoonImage': None, 'voteTarget': {'id': 140444, 'voteServiceId': 15, 'serviceTargetId': 22444, 'voteCount': 16687, 'ranking': 0, 'diff': 0, 'voteTotalScore': 16687, 'dateCreated': '20130916091058', 'lastUpdated': '20191111160011'}, 'shareVoteTarget': {'id': 197673, 'voteServiceId': 23, 'serviceTargetId': 22444, 'voteCount': 28, 'ranking': 0, 'diff': 0, 'voteTotalScore': 28, 'dateCreated': '20141027150913', 'lastUpdated': '201901231

In [6]:
## 완결 웹툰 ##
## 소연아 왜 안했어 ^^

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

soup = readHtml('http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id=&timeStamp=')
str_soup = str(soup)
dict_soup = json.loads(str_soup)
   
toon = dict_soup['data'][0]
print(toon)
# for toon in dict_soup['data']:
#     toon_id = 'daum_' + str(toon['id'])
#     toon_url = 'http://webtoon.daum.net/data/pc/webtoon/view/' + toon['nickname'] + '?timeStamp'
#     toon_name = toon['title']  
#     toon_thumb_url = toon['pcThumbnailImage']['url']
#     toon_site = 'daum'
#     toon_desc = toon['introduction'].replace("  ", " ")
        
#     # 완결 여부
#     if toon['finishYn'] == 'Y':
#         is_end = 'O'
#     else:
#         is_end = 'X'
        
#     # 작가 이름
#     wrt_name_list = list()
#     for names in toon['cartoon']['artists']:
#         wrt_name_list.append(names['name'])
#     wrt_name_list = list(set(wrt_name_list))
#     wrt_name = '/'.join(wrt_name_list)
        
#     # 요일 정보
#     for days in toon['webtoonWeeks']:
#         cursor1.execute(weekday_sql, (toon_id, days['weekDay']))
        
#     # 장르 정보
#     for genres in toon['cartoon']['genres']:
#         genre_name = genres['name'].replace('순정', '로맨스').replace('코믹', '개그').replace('공포', '스릴러')
#         cursor2.execute(genre_sql, (toon_id, genre_name))
        
#     # toon_info
#     cursor3.execute(sel_sql)
#     result = cursor3.fetchall()
#     if (toon_id not in result):
#         cursor4.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url))
        
#     db.commit()
    
# db.close()


{'id': 1824, 'nickname': 'iljin', 'webtoonType': 'series', 'title': '일진', 'finishYn': 'Y', 'thumbnailImage2': {'id': 0, 'url': 'http://t1.daumcdn.net/webtoon/op/6248b92a5f7d7744e6c5653d42f2797a53a06db3'}, 'pcThumbnailImage': {'id': 0, 'url': 'http://t1.daumcdn.net/webtoon/op/5f152d94b4346c190eb4a06660ed5cd39bb52f03'}, 'finishDate': '20191112000000', 'ageGrade': 19, 'dateCreated': '20191024185628', 'cartoon': {'artists': [{'name': '김성모', 'penName': '김성모'}, {'name': '김성모', 'penName': '김성모'}]}, 'latestWebtoonEpisode': {'id': 76684, 'episode': 4, 'title': '4화', 'thumbnailImage': {'id': 76684, 'url': 'http://t1.daumcdn.net/webtoon/bulk/ea2cfdd467e7048f21a97d25755869d7aa15f591'}, 'encryptUseYn': 'N', 'serviceStatus': 'service', 'articleId': 0, 'dateCreated': '20191111113401', 'price': 0, 'isTopRecommend': False, 'isPaid': False, 'ticketAvailable': False, 'ageGrade': 19}, 'webtoonServices': [{'serviceTarget': 'series'}, {'serviceTarget': 'padtoon'}, {'serviceTarget': 'mobile'}, {'serviceTarge

In [ ]:
### 아래부터 지효가 한 완결/에피소드 크롤링 코드

In [70]:
# ------ 2. 완결 웹툰 크롤링
# 에휴 너무 개같아서 안되는거 걍 포기했습니다 누락 작품 700여개 ㅋ

from selenium.common.exceptions import NoSuchElementException

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"

browser = webdriver.Chrome('C:/Users/Administrator/OneDrive/Readers/Crawling/chromedriver.exe')
browser.get('http://m.webtoon.daum.net/m/webtoon/category.html#sort=finish')
browser.implicitly_wait(10)
########## PC버전 완결 탭 속 아이템을 도저히 불러올 방법이 없어서 모바일 버전으로 우회함 ㅋ
######### 대신 썸네일 비율이 쓰레기

page = 1
num = 1
while num < 999999:
    try:
        browser.implicitly_wait(10)
        print(page, num)
        info = browser.find_element_by_xpath("//*[@id=\"daumContent\"]/div[2]/ul["+str(page)+"]/li["+str(num)+"]/a")

        #browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element

        # URL
        toon_url = info.get_attribute('href')
        toon_url = toon_url.replace('m.webtoon', 'webtoon')
        toon_url = toon_url.replace('m/', '')
        
        # ID
        toon_id = 'daum_' + toon_url.replace("http://webtoon.daum.net/webtoon/view/", '')
        
        # 썸네일 주소
        thumb = info.find_element_by_xpath('span[1]/img')
        toon_thumb_url = thumb.get_attribute('data-src')
        
        # 제목
        name = info.find_element_by_class_name('tit')
        toon_name = name.text
        
        # 작가
        name = info.find_element_by_class_name('txt_name')
        wrt_name = name.text
        
        
        toon_desc = ''
        '''
        print(toon_id, toon_name, wrt_name)
        print(toon_url)
        print(toon_thumb_url)
        print("--")
        '''
        # toon_info
        cursor1.execute(sel_sql)
        result = cursor1.fetchall()
        if (toon_id not in result):
            cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'daum', wrt_name, 'O', toon_thumb_url))
        db.commit()
        
        num += 1

    except NoSuchElementException:
        if browser.find_element_by_class_name('btn'):
            browser.find_element_by_class_name('btn').click() # 더보기 버튼 클릭
            page +=1
            num = 1
            continue
        else:
            print("ERROROROORR")
            break
#db.close()

1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1


AttributeError: 'NoneType' object has no attribute 'get_attribute'

In [128]:
# ------ 3. 회차 정보
browser = webdriver.Chrome('C:/Users/Administrator/OneDrive/Readers/Crawling/chromedriver.exe')

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "daum"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

desc_sql = "update toon_info set toon_desc = %s where toon_id = %s and toon_desc = ''"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    browser.get(toon_url)
    browser.implicitly_wait(10)
    
    # 성인 작품 다 삭제했지만.. 아마 19금이어서 안 열리는 애 pass
    if "logins.daum.net" in browser.current_url:
        continue
    
    print('*********************************************')
    print('<' + toon_id + '>')

    #장르
    gn = browser.find_element_by_class_name("txt_genre")
    tag = 1
    while 1:
        try:
            genres = gn.find_element_by_xpath("a["+ str(tag) +"]")
            genre = genres.text
            genre = genre.replace("#", '')
            tag +=1
            cursor2.execute(genre_sql, (toon_id, genre))
        except: break

    #설명
    desc = browser.find_element_by_class_name("txt_story")
    toon_desc = desc.text
    cursor3.execute(desc_sql,(toon_desc, toon_id))
    
    num = 1 # episode 개수 탐색
    page = 1
    while num < 27:
        try:
            epilist = browser.find_element_by_xpath("//*[@id=\"episodeList\"]/ul/li[" + str(num) + "]")

            episode = epilist.find_element_by_xpath("a")
            #browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode

            # URL
            epi_url = episode.get_attribute('href')

            # 썸네일 링크
            thumb = episode.find_element_by_class_name('img_thumb')
            epi_thumb_url = thumb.get_attribute('src')

            # 에피소드 제목
            name = episode.find_element_by_class_name("tit_wt")
            epi_name = name.text

            # 업데이트 날짜
            if epilist.get_attribute('class') == "item_preview":
                #date = epilist.find_element_by_class_name('txt_date') # n일 후 무료 이런 식이라서 디비에 안드감ㅋ
                #date = date.text
                date = "2999-12-31"
            else:
                date = epilist.find_element_by_xpath("div/span")
                if date.get_attribute('class') == "txt_price": # 이거 처리 안해주면 \200 들어감ㅋ
                    date = epilist.find_element_by_xpath("div/span[3]")
                date = date.text
            epi_date = date.replace(".", "-")

            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                pass
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()


            num += 1
            if num == 26:
                page +=1
                browser.get(toon_url + "#pageNo=" + str(page))
                num = 1

        except:
            #print("!!!!!!! EROROR")
            break
        
db.close()

*********************************************
<daum_1047>
*********************************************
<daum_107>
*********************************************
<daum_1078>
*********************************************
<daum_1133>
*********************************************
<daum_1185>
*********************************************
<daum_1190>
*********************************************
<daum_11yearafterour>
*********************************************
<daum_1229>
*********************************************
<daum_1258>
*********************************************
<daum_1273>


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1273-전생' for key 'PRIMARY'")
  self._do_get_result()


*********************************************
<daum_1291>


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1291-갱스터' for key 'PRIMARY'")
  self._do_get_result()


*********************************************
<daum_1295>


...

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_869-판타지' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_869-순정' for key 'PRIMARY'")
  self._do_get_result()


*********************************************
<daum_975>


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_975-드라마' for key 'PRIMARY'")
  self._do_get_result()


*********************************************
<daum_againflower>
*********************************************
<daum_airlove>
*********************************************
<daum_angelhouse>
*********************************************
<daum_animalhero>
*********************************************
<daum_artist>
*********************************************
<daum_backeunaran>
*********************************************
<daum_bbiguraebong>
*********************************************
<daum_beltupdown>
*********************************************
<daum_bindongbindong>
*********************************************
<daum_blackark>
*********************************************
<daum_blinker>
*********************************************
<daum_bliznitch>
*********************************************
<daum_bluffingguy>
*********************************************
<daum_bodyforrent>
*********************************************
<daum_butterflydream>
***************************************

*********************************************
<daum_rebutdidi>
*********************************************
<daum_relationship>
*********************************************
<daum_romeoisjuliet>
*********************************************
<daum_rooftopdrift>
*********************************************
<daum_runstarmoon>
*********************************************
<daum_sadstop>
*********************************************
<daum_sangod>
*********************************************
<daum_sasaeng>
*********************************************
<daum_seoulknight>
*********************************************
<daum_shadowboy>
*********************************************
<daum_sheisfull>
*********************************************
<daum_sheisvip>
*********************************************
<daum_sherlockqueen>
*********************************************
<daum_shimchung>
*********************************************
<daum_shivering>
*********************************************